<a href="https://colab.research.google.com/github/ageofthegit/trading-local/blob/main/CT01_2_Bars_since.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Strategy :**
Simple RSI based trading strategy for BTC

**Backtest :**
Backtested over 2020/2023



*CT01.1_Trading.ipynb does *

Ideas:
- Long + Shor on RSI
- Testing Bars since last exit for Long RSI (works!)


*CT01_Trading.ipynb does *

Ideas:
- Shorting on RSI
- Stop loss on RSI  



In [ ]:
# Mounting Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# !pip install vectorbt

In [ ]:
# Importing Librarie(s)

import pandas as pd, numpy as np
import datetime, pytz
import vectorbt as vbt

local_tz = pytz.timezone('Asia/Kolkata')  # Replace with your time zone

#from sktime import sliding_window_view



In [ ]:
# !pip install sktime

In [ ]:

#file_path = "/content/drive/Computers/Omen/Gdrive/data-quant/BTC"
file_path = "/content/drive/MyDrive/Quantvaala/data"
model_path = "/content/drive/MyDrive/Quantvaala/model"

logs_path = "/content/drive/MyDrive/Quantvaala/logs"

symbol = "BTC"
model_path = model_path + '/' + symbol
file_path = file_path + '/' + symbol

ctime = "15m"

vars_basc = {symbol : symbol, ctime : ctime}
vars_back = {}
vars_strt = {}
vars_adnl = {}

file1 = "data_2020_2023_15min.csv"
# file2 = "data_2020_2023_15min.csv"

pricesdf1 = pd.read_csv(f"{file_path}/{file1}", index_col=0)
print(pricesdf1.columns.values.tolist())
print(pricesdf1.shape)

# pricesdf2 = pd.read_csv(f"{file_path}/{file2}", index_col=0)
# print(pricesdf2.columns.values.tolist())
# print(pricesdf2.shape)


# print(f"Importing {file1} for {symbol}")
# pricesdf = pd.read_csv(f"{file_path}", index_col=0)

# Use pd.concat to combine them
print()
#pricesdf = pd.concat( [pricesdf1, pricesdf2], ignore_index=True )

# pricesdf = pd.concat( [pricesdf1, pricesdf2])
pricesdf = pricesdf1.copy()
# pricesdf[['open','high','low','close']] = pricesdf[['open','high','low','close']]#.astype('float64')
pricesdf[['open','high','low','close']] = pricesdf[['open','high','low','close']] / 100000


print(pricesdf.columns.values.tolist())
print(pricesdf.shape)

del pricesdf1


['open', 'high', 'low', 'close', 'volume']
(140104, 5)

['open', 'high', 'low', 'close', 'volume']
(140104, 5)


In [ ]:

display(pricesdf.head())
display(pricesdf.tail())


,open,high,low,close,volume
datetime,,,,,
2020-01-01 05:30:00,0.071952,0.071962,0.071782,0.071810,202.942868
2020-01-01 05:45:00,0.071810,0.071864,0.071755,0.071785,128.242654
2020-01-01 06:00:00,0.071782,0.071854,0.071762,0.071796,83.487458
2020-01-01 06:15:00,0.071794,0.071840,0.071755,0.071770,97.141921
2020-01-01 06:30:00,0.071765,0.071940,0.071757,0.071909,103.520522


,open,high,low,close,volume
datetime,,,,,
2024-01-01 04:30:00,0.422579,0.423457,0.421966,0.422301,637.99923
2024-01-01 04:45:00,0.422301,0.422351,0.420660,0.421818,614.77027
2024-01-01 05:00:00,0.421818,0.422429,0.421543,0.422411,215.21045
2024-01-01 05:15:00,0.422411,0.422836,0.422212,0.422836,242.15726
2024-01-01 05:30:00,0.422836,0.424881,0.422610,0.424880,431.71082


In [ ]:

pricesdf.dtypes


In [ ]:

# Generating Train x Test Split at 80% and 20%

pricesdf.index = pd.to_datetime(pricesdf.index, format='%Y-%m-%d %H:%M:%S')

# columns_for_prediction = ["close","volume"]

# data_train = pd.DataFrame(pricesdf[columns_for_prediction][0: int(len(pricesdf)*0.80)])
# print(data_train.shape)
# display(data_train.tail(5))

# data_test = pd.DataFrame(pricesdf[columns_for_prediction][int(len(pricesdf)*0.80): len(pricesdf)])
# print(data_test.shape)
# display(data_test.tail(5))


In [ ]:

%%time

# import pandas as pd
# import numpy as np

# Load your OHLC data into a pandas DataFrame
# ohlc_data = pd.read_csv('bitcoin_ohlc_15m.csv', parse_dates=['datetime'])
ohlc_data = pricesdf.copy()

# Set the start and end dates for backtesting
start_date = '2020-01-01'  # Customize the start date
end_date = '2023-12-31'  # Customize the end date

vars_back['dt_strt'] = start_date
vars_back['dt_ends'] = end_date

# Filter the data based on the specified time frame
# ohlc_data = ohlc_data.loc[(ohlc_data['datetime'] >= start_date) & (ohlc_data['datetime'] <= end_date)]
ohlc_data = ohlc_data.loc[(ohlc_data.index >= start_date) & (ohlc_data.index <= end_date)]
print(f"\t shape of ohlc_data is {ohlc_data.shape}")

# Convert the DataFrame to a vectorbt.Vector
# price = vbt.Vector.from_pandas(ohlc_data['close'])
price = ohlc_data['close'].values

# Calculate the RSI
rsi_period = 14
# rsi = vbt.indicators.RSI(price, window=period)
rsi = vbt.RSI.run(ohlc_data.close, window = rsi_period, short_name="rsi")

vars_strt['strat'] = 'rsi'
vars_strt['rsi_period'] = rsi_period

# Set the trading parameters
rsi_overbought = 70
rsi_oversold = 30
# stop_loss = 0.98  # 2% stop loss
# take_profit = 1.03  # 3% take profit

vars_strt['rsi_ob'] = rsi_overbought
vars_strt['rsi_os'] = rsi_oversold

# Generate the entry and exit signals
# entries = rsi.vbt.signals.cross_below(rsi_oversold)
exits_raw = rsi.rsi_crossed_above( rsi_overbought )
entries_raw = rsi.rsi_crossed_below( rsi_oversold )

# Set the stop-loss and take-profit levels
# stop_loss_levels = price * stop_loss
# take_profit_levels = price * take_profit



	 shape of ohlc_data is (139986, 5)
CPU times: user 5.78 s, sys: 65.5 ms, total: 5.85 s
Wall time: 5.84 s


In [ ]:
print(vars_basc, vars_back, vars_strt)

In [ ]:

print(type(entries_raw))
print(len(entries_raw))
print(entries_raw[:5])

print()
print(type(exits_raw))
print(len(exits_raw))
print(exits_raw[:5])


In [ ]:

# print("## Implementing a Cooling off period for entries")

# print("# Define the minimum number of bars to wait after an entry before considering new entries")
# bars_since_last_entry_threshold = 10  # Example: 5 bars

# vars_adnl['bars_since_lst_entry'] = bars_since_last_entry_threshold

# print("# Initialize an array to hold the modified entry signals")
# print(entries_raw.shape)
# modified_entries = np.full(entries_raw.shape, False)

# print("# Loop through the entry signals")
# last_entry_index = None
# for i in range(len(entries_raw)):
#     if entries_raw[i]:
#         if last_entry_index is None or i - last_entry_index > bars_since_last_entry_threshold:
#             # Set the entry signal
#             modified_entries[i] = True
#             # Update the last entry index
#             last_entry_index = i


# print("# Initialize an array to hold the adjusted exit signals")
# exits = np.full(exits_raw.shape, False)

# print("# Track whether we're in a trade")
# in_trade = False

# for i in range(len(exits_raw)):
#     if modified_entries[i]:
#         # An entry signal - we're now in a trade
#         in_trade = True
#     elif exits_raw[i] and in_trade:
#         # An exit signal and we're in a trade
#         exits[i] = True
#         in_trade = False  # Exiting the trade


# print("# Replace the original entries with the modified ones")
# entries = modified_entries

# print("# Clean-up")
# del modified_entries, in_trade, bars_since_last_entry_threshold


In [ ]:


# print("# Define the minimum number of bars to wait after an EXIT before re-entry")
bars_since_last_exit = 10  # Example: 5 bars
vars_adnl['bars_since_last_exit'] = bars_since_last_exit

print("# Initialize an array to hold the modified entry signals")
print(entries_raw.shape)
modified_entries = np.full(entries_raw.shape, False)

print("# Initialize the last exit index variable")
last_exit_index = None

print("# Loop through the signals to modify entries based on last exit")
for i in range(len(entries_raw)):
    if exits_raw[i]:
        # Update the last exit index when an exit occurs
        last_exit_index = i
    elif entries_raw[i]:
        # Check if we're beyond the cooling off period since the last exit
        if last_exit_index is None or i - last_exit_index > bars_since_last_exit:
            # Set the modified entry signal
            modified_entries[i] = True

print("# Initialize an array to hold the adjusted exit signals")
exits = np.full(exits_raw.shape, False)

print("# Track whether we're in a trade")
in_trade = False

for i in range(len(exits_raw)):
    if modified_entries[i]:
        # An entry signal - we're now in a trade
        in_trade = True
    elif exits_raw[i] and in_trade:
        # An exit signal and we're in a trade
        exits[i] = True
        in_trade = False  # Exiting the trade
        # Update the last exit index
        last_exit_index = i

print("# Replace the original entries with the modified ones")
entries = modified_entries

print("# Clean-up")
del modified_entries, in_trade, bars_since_last_exit, last_exit_index


# Initialize an array to hold the modified entry signals
(139986,)
# Initialize the last exit index variable
# Loop through the signals to modify entries based on last exit
# Initialize an array to hold the adjusted exit signals
# Track whether we're in a trade
# Replace the original entries with the modified ones
# Clean-up


In [ ]:


# print("# Define the minimum number of bars to wait after an EXIT before re-entry")
bars_since_last_cover = 10  # Example: 5 bars
vars_adnl['bars_since_last_cover'] = bars_since_last_cover

entries_short_raw = exits_raw.copy()
exits_short_raw = entries_raw.copy()

print("# Initialize an array to hold the modified entry signals")
print(entries_short_raw.shape)
modified_entries_short = np.full(entries_short_raw.shape, False)

print("# Initialize the last exit index variable")
last_cover_index = None

print("# Loop through the signals to modify SHORT entries based on last COVER")
for i in range(len(entries_short_raw)):
    if exits_short_raw[i]:
        # Update the last exit index when an exit occurs
        last_cover_index = i
    elif entries_short_raw[i]:
        # Check if we're beyond the cooling off period since the last exit
        if last_cover_index is None or i - last_cover_index > bars_since_last_cover:
            # Set the modified entry signal
            modified_entries_short[i] = True

print("# Initialize an array to hold the adjusted exit signals")
exits_short = np.full(exits_short_raw.shape, False)

print("# Track whether we're in a trade")
in_trade_short = False

for i in range(len(exits_short_raw)):
    if modified_entries_short[i]:
        # An entry signal - we're now in a trade
        in_trade_short = True
    elif exits_short_raw[i] and in_trade_short:
        # An exit signal and we're in a trade
        exits_short[i] = True
        in_trade_short = False  # Exiting the trade
        # Update the last exit index
        last_cover_index = i

print("# Replace the original entries with the modified ones")
entries_short = modified_entries_short

print("# Clean-up")
del modified_entries_short, in_trade_short, bars_since_last_cover, last_cover_index


# Initialize an array to hold the modified entry signals
(139986,)
# Initialize the last exit index variable
# Loop through the signals to modify SHORT entries based on last COVER
# Initialize an array to hold the adjusted exit signals
# Track whether we're in a trade
# Replace the original entries with the modified ones
# Clean-up


In [ ]:

# # COMBINING THE CODE

# # Define the minimum number of bars to wait after an EXIT before re-entry for both long and short positions
# bars_since_last_exit = 10  # For long positions
# bars_since_last_cover = 10  # For short positions
# vars_adnl['bars_since_last_exit'] = bars_since_last_exit
# vars_adnl['bars_since_last_cover'] = bars_since_last_cover

# # Initialize arrays to hold the modified entry and exit signals for both long and short positions
# modified_entries = np.full(entries_raw.shape, False)
# modified_entries_short = np.full(exits_raw.shape, False)  # Using entries_raw shape for simplicity
# exits = np.full(exits_raw.shape, False)
# exits_short = np.full(entries_raw.shape, False)

# # Initialize the last exit and cover index variables
# last_exit_index = None
# last_cover_index = None

# # Initialize flags to track whether we're in a trade (long or short)
# in_trade = False
# in_trade_short = False

# # Loop through the signals to modify entries and exits based on the logic
# for i in range(len(entries_raw)):
#     if in_trade:
#         # If in a long trade, check for exit
#         if exits_raw[i]:
#             exits[i] = True
#             in_trade = False
#             last_exit_index = i
#             in_trade_short = False  # Ensure we exit short positions when a long exit occurs
#     elif in_trade_short:
#         # If in a short trade but a long entry comes in, prioritize the long entry
#         if entries_raw[i]:
#             modified_entries[i] = True
#             in_trade = True
#             in_trade_short = False  # Exit short position
#             last_exit_index = i
#         elif exits_short_raw[i]:
#             # Process short exits
#             exits_short[i] = True
#             in_trade_short = False
#             last_cover_index = i
#     else:
#         # If not in any trade, check for long entries first
#         if entries_raw[i] and (last_exit_index is None or i - last_exit_index > bars_since_last_exit):
#             modified_entries[i] = True
#             in_trade = True
#         # If no long entry, check for short entries
#         elif entries_short_raw[i] and (last_cover_index is None or i - last_cover_index > bars_since_last_cover):
#             modified_entries_short[i] = True
#             in_trade_short = True

# # Replace the original entries with the modified ones for both long and short
# entries = modified_entries
# entries_short = modified_entries_short

# # Clean-up
# del modified_entries, modified_entries_short
# del in_trade, in_trade_short, bars_since_last_exit, bars_since_last_cover, last_exit_index, last_cover_index



In [ ]:
print(vars_basc, vars_back, vars_strt, vars_adnl)

{'BTC': 'BTC', '15m': '15m'} {'dt_strt': '2020-01-01', 'dt_ends': '2023-12-31'} {'strat': 'rsi', 'rsi_period': 14, 'rsi_ob': 70, 'rsi_os': 30} {'bars_since_last_exit': 10, 'bars_since_last_cover': 10}


In [ ]:

%%time

# Create a portfolio
# pf = vbt.Portfolio.from_signals(price, entries, exits, stop_loss=stop_loss_levels, take_profit=take_profit_levels)
pf = vbt.Portfolio.from_signals(price,
          entries = entries,
          exits = exits,
          short_entries = entries_short,
          short_exits = exits_short,
          freq = '15T')

# Print the performance metrics
stats = pf.stats()
# stats = pf.stats(frequency='15T') # errors
# stats = pf.stats(freq='15T') # errors
print(stats)
print()

# Backtest the strategy
# pf.plot().show()



Start                                                 0
End                                              139985
Period                               1458 days 04:30:00
Start Value                                       100.0
End Value                                    202.100647
Total Return [%]                             102.100647
Benchmark Return [%]                         489.629395
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              52.464735
Max Drawdown Duration                 362 days 00:45:00
Total Trades                                       2925
Total Closed Trades                                2924
Total Open Trades                                     1
Open Trade PnL                                -1.274459
Win Rate [%]                                  67.270862
Best Trade [%]                                13.547798
Worst Trade [%]                              -20

In [ ]:

statsdf_raw = stats.to_frame()
# print( statsdf_raw.index )
print( statsdf_raw.columns.values.tolist() )

# Convert dictionary to a Series, the name 'NewRow' will be the index of the new row
# s_basc = pd.Series(vars_basc, name='vars_basc')
print(vars_basc)
df_basc = pd.DataFrame([str(vars_basc)], index=['vars_basc'])
df_back = pd.DataFrame([str(vars_back)], index=['vars_back'])
df_strt = pd.DataFrame([str(vars_strt)], index=['vars_strt'])
df_adnl = pd.DataFrame([str(vars_adnl)], index=['vars_adnl'])

# Append the new row to the DataFrame
# statsdf = statsdf.append(df_basc)
# statsdf = pd.concat([statsdf_raw, s_basc], ignore_index=False)
statsdf = pd.concat([statsdf_raw, df_basc, df_back, df_strt, df_adnl], ignore_index=False)

display( statsdf )


[0]
{'BTC': 'BTC', '15m': '15m'}


,0
Start,0
End,139985
Period,1458 days 04:30:00
Start Value,100.0
End Value,202.100647
Total Return [%],102.100647
Benchmark Return [%],489.629395
Max Gross Exposure [%],100.0
Total Fees Paid,0.0
Max Drawdown [%],52.464735


In [ ]:

# statsdf.to_excel(f"{logs_path}/combined_logs_file.xlsx")


In [ ]:

# Read the existing file
comblogs_df = pd.read_excel(f"{logs_path}/combined_logs_file.xlsx", index_col=0)
# existing_df = pd.read_csv('existing_file.csv', index_col=0)

# Create a new DataFrame with the desired column
# new_column = [value1, value2, value3, ...]
# new_df = pd.DataFrame(new_column, index=existing_df.index, columns=['New Column Name'])

# Concatenate the new column with the existing DataFrame
new_combologs_df = pd.concat([comblogs_df, statsdf], axis=1)

# Write to a new Excel file
new_combologs_df.to_excel(f'{logs_path}/combined_logs_file.xlsx')

# Write to a new CSV file
# combined_df.to_csv('output_file.csv')


In [ ]:


# print()

# display(pf_trade.describe())

# print()
#
# display(pf_trade.head(5))




In [ ]:

%%time

pf_trade = pf.trades.records

# The location of the pf_trade
print(pf_trade.columns.values.tolist())
print(pf_trade.shape)

# Initialize the 'trade_action' column to 0 for no action
print(ohlc_data.shape)
ohlc_exp = ohlc_data.copy().reset_index()
print(ohlc_exp.columns.values.tolist())
ohlc_exp['trade_action'] = 0

# Mark trade entries - you can replace 1 with any identifier you prefer for trade entries
ohlc_exp.loc[pf_trade['entry_idx'].astype(int), 'trade_action'] = 1

# Mark trade exits - you can replace -1 with any identifier you prefer for trade exits
# Ensure there's no overlap between entries and exits, or exits will overwrite entries in 'trade_action'
ohlc_exp.loc[pf_trade['exit_idx'].astype(int), 'trade_action'] = -1




['id', 'col', 'size', 'entry_idx', 'entry_price', 'entry_fees', 'exit_idx', 'exit_price', 'exit_fees', 'pnl', 'return', 'direction', 'status', 'parent_id']
(2925, 14)
(139986, 5)
['datetime', 'open', 'high', 'low', 'close', 'volume']
CPU times: user 9.66 ms, sys: 3.08 ms, total: 12.7 ms
Wall time: 11.3 ms


In [ ]:

print(type(ohlc_exp))
print(ohlc_exp.shape)
print(ohlc_exp.columns.values.tolist())

display(ohlc_exp.head(5))
print()

display(ohlc_exp.describe())


<class 'pandas.core.frame.DataFrame'>
(139986, 7)
['datetime', 'open', 'high', 'low', 'close', 'volume', 'trade_action']


,datetime,open,high,low,close,volume,trade_action
0,2020-01-01 05:30:00,0.071952,0.071962,0.071782,0.071810,202.942868,0
1,2020-01-01 05:45:00,0.071810,0.071864,0.071755,0.071785,128.242654,0
2,2020-01-01 06:00:00,0.071782,0.071854,0.071762,0.071796,83.487458,0
3,2020-01-01 06:15:00,0.071794,0.071840,0.071755,0.071770,97.141921,0
4,2020-01-01 06:30:00,0.071765,0.071940,0.071757,0.071909,103.520522,0


,datetime,open,high,low,close,volume,trade_action
count,139986,139986.000000,139986.000000,139986.000000,139986.000000,139986.000000,139986.000000
mean,2021-12-31 12:42:50.174874880,0.288419,0.289113,0.287703,0.288421,1009.280326,-0.018009
min,2020-01-01 05:30:00,0.038886,0.042408,0.037821,0.038822,0.000000,-1.000000
25%,2020-12-31 15:18:45,0.169271,0.169432,0.169073,0.169271,298.147241,0.000000
50%,2021-12-31 20:37:30,0.273012,0.273484,0.272510,0.273019,539.127942,0.000000
75%,2022-12-31 09:41:15,0.397026,0.397986,0.395925,0.397028,1145.150429,0.000000
max,2023-12-31 00:00:00,0.687189,0.690000,0.684800,0.687189,40371.405060,1.000000
std,NaN,0.150991,0.151429,0.150539,0.150990,1398.922980,0.153156


In [ ]:


columns_to_rescale = ['open','high','low','close']
ohlc_exp [columns_to_rescale] = ohlc_exp [columns_to_rescale]*100000

ohlc_exp.to_excel(f'{logs_path}/basic_ohlc_exp.xlsx')



In [ ]:


# Convert the trades to a DataFrame
trades_df = pd.DataFrame({
    'Entry_Time': pf.trades.entry_time,
    'Exit_Time': pf.trades.exit_time,
    'PnL': pf.trades.pnl.values
})

# Convert the timestamps to datetime objects
trades_df['Entry_Time'] = pd.to_datetime(trades_df['Entry_Time'])
trades_df['Exit_Time'] = pd.to_datetime(trades_df['Exit_Time'])

# Extract the month and year from the Entry_Time column
trades_df['Month_Year'] = trades_df['Entry_Time'].dt.to_period('M')

# Calculate the monthly profits and losses
monthly_pnl = trades_df.groupby('Month_Year')['PnL'].sum()

# Print the monthly profits and losses
print(monthly_pnl)


AttributeError: 'ExitTrades' object has no attribute 'entry_time'

In [ ]:


# Create a portfolio
# pf = vbt.Portfolio.from_signals(price, entries, exits, stop_loss=stop_loss_levels, take_profit=take_profit_levels)
pf_short = vbt.Portfolio.from_signals( price, short_exits = entries, short_entries = exits)

# Print the performance metrics
print(pf_short.stats())
print()

# Backtest the strategy
# pf_short.plot().show()


In [ ]:


# Create a portfolio
# pf = vbt.Portfolio.from_signals(price, entries, exits, stop_loss=stop_loss_levels, take_profit=take_profit_levels)
pf_short_wsl = vbt.Portfolio.from_signals( price, short_exits = entries, short_entries = exits, sl_stop = 0.1)

# Backtest the strategy
pf_short_wsl.plot().show()

# Print the performance metrics
print()
# print(pf_short_wsl.stats())


In [ ]:

# Create a portfolio
# pf = vbt.Portfolio.from_signals(price, entries, exits, stop_loss=stop_loss_levels, take_profit=take_profit_levels)
pf = vbt.Portfolio.from_signals(price, entries, exits, sl_stop = 0.1)

# Print the performance metrics
print(pf.stats())
print()

# Backtest the strategy
# pf.plot().show()



In [ ]:

# Create a portfolio
# pf = vbt.Portfolio.from_signals(price, entries, exits, stop_loss=stop_loss_levels, take_profit=take_profit_levels)
pf = vbt.Portfolio.from_signals(price, entries, exits, sl_stop = 0.05, sl_trail = True)

# Print the performance metrics
print(pf.stats())
print()

# Backtest the strategy
# pf.plot().show()

